# Formula/Function

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm

##Black-Schole

In [ ]:
def Black_Schole(spot_price, strike_price, dividend_yield, risk_free_rate,time_to_maturity):
  d1 = (np.log(spot_price / strike_price) + (dividend_yield + (volatility ** 2) / 2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
  d2 = d1 - volatility * np.sqrt(time_to_maturity)
  N_d1 = norm.cdf(d1)
  N_d2 = norm.cdf(d2)
  call_option_price = spot_price * np.exp((dividend_yield - risk_free_rate) * time_to_maturity) * norm.cdf(d1) - strike_price * np.exp(-risk_free_rate * time_to_maturity) * norm.cdf(d2)
  call_option_delta = np.exp((dividend_yield - risk_free_rate) * time_to_maturity) * norm.cdf(d1)
  put_option_price = strike_price * np.exp(-risk_free_rate * time_to_maturity) * norm.cdf(-d2) - spot_price * np.exp((dividend_yield-risk_free_rate) * time_to_maturity) * norm.cdf(-d1)
  put_option_delta = -np.exp((dividend_yield-risk_free_rate)*time_to_maturity) * norm.cdf(-d1)
  return d1, d2, N_d1, N_d2, call_option_price, call_option_delta, put_option_price, put_option_delta

##Binimial Tree

In [ ]:
import numpy as np

def generate_asset_price(spot_price, d, u, num_steps):
    x = np.zeros((num_steps + 1, num_steps + 1))

    for i in range(num_steps + 1):
        for j in range(num_steps + 1):
            if i == 0 and j == 0:
                x[i, j] = spot_price
            elif i == j:
                x[i, j] = x[i-1, j-1] * d
            elif i < j:
                x[i, j] = x[i, j-1] * u
            else:
                x[i, j] = 0

    return x

In [ ]:
import numpy as np

def generate_european_call(spot_price, strike_price, d, u, q, r, num_steps):
    x = np.zeros((num_steps + 1, num_steps + 1))
    y = generate_asset_price(spot_price, d, u, num_steps)

    for i in range(num_steps + 1):
        for j in range(num_steps + 1):
            if j == num_steps:
                x[i, j] = max(0, y[i, j] - strike_price)

    n = 1
    for k in range(num_steps + 1 - n):
        for i in range(num_steps -n +1):
            for j in range(num_steps):
                if j <= num_steps - 1 and i <= num_steps - 1and i <= j:
                    x[i, j] = (q * x[i, j+1] + (1 - q) * x[i+1, j+1]) / r
        n += 1
    return x

In [ ]:
import numpy as np

def generate_american_call(spot_price, strike_price, d, u, q, r, num_steps):
    x = np.zeros((num_steps + 1, num_steps + 1))
    y = generate_asset_price(spot_price, d, u, num_steps)

    for i in range(num_steps + 1):
        for j in range(num_steps + 1):
            if j == num_steps:
                x[i, j] = max(0, y[i, j] - strike_price)

    n = 1
    for k in range(num_steps + 1 - n):
        for i in range(num_steps -n + 1):
            for j in range(num_steps):
                if j <= num_steps - 1 and i <= num_steps - 1 and i <= j:
                    x[i, j] = np.maximum(y[i,j] -  strike_price,(q * x[i, j+1] + (1 - q) * x[i+1, j+1]) / r)

        n += 1
    return x

In [ ]:
import numpy as np

def generate_european_put(spot_price, strike_price, d, u, q, r, num_steps):
    x = np.zeros((num_steps + 1, num_steps + 1))
    y = generate_asset_price(spot_price, d, u, num_steps)

    for i in range(num_steps + 1):
        for j in range(num_steps + 1):
            if j == num_steps:
                x[i, j] = max(0, strike_price - y[i, j])

    n = 1
    for k in range(num_steps + 1 - n):
        for i in range(num_steps -n +1):
            for j in range(num_steps):
                if j <= num_steps - 1 and i <= num_steps - 1and i <= j:
                    x[i, j] = (q * x[i, j+1] + (1 - q) * x[i+1, j+1]) / r
        n += 1
    return x

In [ ]:
import numpy as np


def generate_american_put(spot_price, strike_price, d, u, q, r, num_steps):
    x = np.zeros((num_steps + 1, num_steps + 1))
    y = generate_asset_price(spot_price, d, u, num_steps)

    for i in range(num_steps + 1):
        for j in range(num_steps + 1):
            if j == num_steps:
                x[i, j] = max(0, strike_price - y[i, j])

    n = 1
    for k in range(num_steps + 1 - n):
        for i in range(num_steps -n + 1):
            for j in range(num_steps):
                if j <= num_steps - 1 and i <= num_steps - 1 and i <= j:
                    x[i, j] = np.maximum(strike_price - y[i,j],(q * x[i, j+1] + (1 - q) * x[i+1, j+1]) / r)

        n += 1
    return x

##Monte Carlo

In [ ]:
import numpy as np
from scipy.stats import norm

def monte_carlo_sim_option(spot_price, risk_free_rate, dividend_yield, volatility, time_to_maturity, num_paths):
  simulation = np.zeros((num_paths + 1, 1))

  for i in range(1, num_paths + 1):
    drift = (risk_free_rate - dividend_yield - 0.5 * volatility**2) * time_to_maturity
    vol_term = volatility * np.sqrt(time_to_maturity)
    random_number = np.random.rand()
    sim_price = spot_price * np.exp(drift + vol_term * norm.ppf(random_number))

    simulation[i-1, 0] = sim_price

  return simulation

# Run Option Value

## Option Input

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
# Example usage:
spot_price = 210   # Current stock price
strike_price = 175   # Option strike price
time_to_maturity = 5  # Time to option expiration in years
volatility = np.sqrt(0.2)  # Volatility of the underlying stock
risk_free_rate = 0.04 # Risk-free interest rate
dividend_yield =  risk_free_rate - 0.029


##Black-Schole

In [ ]:
d1, d2, N_d1, N_d2, call_option_price, call_option_delta, put_option_price, put_option_delta = Black_Schole(spot_price, strike_price, dividend_yield, risk_free_rate,time_to_maturity)

In [ ]:
print(f"d1 is {d1:.3f}\nd2 is {d2:.3f}")
print(f"N(d1) is {N_d1:.3f}\nN(d2) is {N_d2:.3f}")
print(f"Call Value is {call_option_price:.3f}\nCall Delta is {call_option_delta:.3f}")
print(f"Put Value is {put_option_price:.3f}\nPut Delta is {put_option_delta:.3f}")

d1 is 0.737
d2 is -0.263
N(d1) is 0.770
N(d2) is 0.396
Call Value is 82.995
Call Delta is 0.666
Put Value is 44.618
Put Delta is -0.199


##Binomial Tree

In [ ]:
import numpy as np
"""
spot_price = 274      # Current stock price
strike_price = 225   # Option strike price
time_to_maturity = 32/252  # Time to option expiration in years
volatility = 0.52    # Volatility of the underlying stock
risk_free_rate = 0.01 # Risk-free interest rate
dividend_yield = 0.00
"""
num_steps = 60


delta_t = time_to_maturity / num_steps
u = np.exp(volatility * np.sqrt(delta_t))
d = 1 / u
r = np.exp(risk_free_rate*delta_t)
b = np.exp(dividend_yield*delta_t)
q = (b-d)/(u-d)

In [ ]:
Asset_Price = generate_asset_price(spot_price, d, u, num_steps)
Asset_Price_df = pd.DataFrame(Asset_Price)
Asset_Price_df = Asset_Price_df.round(2)
#Asset_Price_df.replace(0, '', inplace=True)
#Asset_Price_df.head(num_steps+1)
Asset_Price_df

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,210.0,238.94,271.87,309.33,351.96,400.46,455.64,518.43,589.87,671.16,...,151930.62,172867.15,196688.81,223793.17,254632.60,289721.80,329646.41,375072.77,426759.02,485567.81
1,0.0,184.57,210.00,238.94,271.87,309.33,351.96,400.46,455.64,518.43,...,117357.53,133529.78,151930.62,172867.15,196688.81,223793.17,254632.60,289721.80,329646.41,375072.77
2,0.0,0.00,162.21,184.57,210.00,238.94,271.87,309.33,351.96,400.46,...,90651.83,103143.95,117357.53,133529.78,151930.62,172867.15,196688.81,223793.17,254632.60,289721.80
3,0.0,0.00,0.00,142.57,162.21,184.57,210.00,238.94,271.87,309.33,...,70023.25,79672.68,90651.83,103143.95,117357.53,133529.78,151930.62,172867.15,196688.81,223793.17
4,0.0,0.00,0.00,0.00,125.30,142.57,162.21,184.57,210.00,238.94,...,54088.87,61542.49,70023.25,79672.68,90651.83,103143.95,117357.53,133529.78,151930.62,172867.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.15,0.17,0.20,0.22,0.26
57,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.13,0.15,0.17,0.20
58,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.12,0.13,0.15
59,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.10,0.12


In [ ]:
euro_call = generate_european_call(spot_price, strike_price ,d, u, q, r ,num_steps)
euro_call_df = pd.DataFrame(euro_call)
euro_call_df = euro_call_df.round(2)
euro_call_df.replace(0, '', inplace=True)
euro_call_df.head(num_steps+1)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,83.13,102.6,125.87,153.53,186.22,224.65,269.61,321.97,382.72,452.95,...,148491.97,169386.76,193218.51,220400.05,251402.2,286761.97,327091.86,373090.45,425554.51,485392.81
1,,66.3,82.5,102.01,125.36,153.14,185.99,224.63,269.85,322.52,...,114662.73,130802.6,149211.11,170207.13,194154.35,221467.58,252619.93,288151.02,328676.31,374897.77
2,,,52.28,65.62,81.83,101.39,124.82,152.72,185.73,224.59,...,88531.62,100998.59,115217.96,131436.03,149933.73,171031.47,195094.71,222540.28,253843.56,289546.8
3,,,,40.72,51.59,64.92,81.14,100.73,124.25,152.27,...,68346.86,77976.74,88960.24,101487.62,115775.88,132072.52,150659.84,171859.8,196039.63,223618.17
4,,,,,31.28,40.03,50.87,64.18,80.41,100.05,...,52755.31,60193.7,68677.69,78354.22,89390.94,101979.02,116336.5,132712.1,151389.48,172692.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,,,,,,,,,,,...,,,,,,,,,,
57,,,,,,,,,,,...,,,,,,,,,,
58,,,,,,,,,,,...,,,,,,,,,,
59,,,,,,,,,,,...,,,,,,,,,,


In [ ]:
am_call = generate_american_call(spot_price, strike_price ,d, u, q, r ,num_steps)
am_call_df = pd.DataFrame(am_call)
am_call_df = am_call_df.round(2)
am_call_df.replace(0, '', inplace=True)
am_call_df.head(num_steps+1)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,86.59,107.26,132.13,161.89,197.33,239.36,288.99,347.4,415.93,496.16,...,151755.62,172692.15,196513.81,223618.17,254457.6,289546.8,329471.41,374897.77,426584.02,485392.81
1,,68.71,85.77,106.43,131.32,161.12,196.63,238.74,288.49,347.04,...,117182.53,133354.78,151755.62,172692.15,196513.81,223618.17,254457.6,289546.8,329471.41,374897.77
2,,,53.94,67.89,84.93,105.58,130.48,160.32,195.9,238.12,...,90476.83,102968.95,117182.53,133354.78,151755.62,172692.15,196513.81,223618.17,254457.6,289546.8
3,,,,41.84,53.14,67.05,84.05,104.7,129.62,159.51,...,69848.25,79497.68,90476.83,102968.95,117182.53,133354.78,151755.62,172692.15,196513.81,223618.17
4,,,,,32.03,41.07,52.31,66.17,83.15,103.79,...,53913.87,61367.49,69848.25,79497.68,90476.83,102968.95,117182.53,133354.78,151755.62,172692.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,,,,,,,,,,,...,,,,,,,,,,
57,,,,,,,,,,,...,,,,,,,,,,
58,,,,,,,,,,,...,,,,,,,,,,
59,,,,,,,,,,,...,,,,,,,,,,


In [ ]:
euro_put = generate_european_put(spot_price, strike_price ,d, u, q, r ,num_steps)
euro_put_df = pd.DataFrame(euro_put)
euro_put_df = euro_put_df.round(2)
euro_put_df.replace(0, '', inplace=True)
euro_put_df.head(num_steps+1)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,44.75,39.16,33.8,28.72,24.01,19.72,15.88,12.53,9.66,7.26,...,,,,,,,,,,
1,,50.02,44.2,38.53,33.1,27.99,23.26,18.97,15.16,11.85,...,,,,,,,,,,
2,,,55.52,49.53,43.61,37.87,32.38,27.23,22.49,18.21,...,,,,,,,,,,
3,,,,61.22,55.11,49.01,43.0,37.18,31.64,26.45,...,,,,,,,,,,
4,,,,,67.04,60.9,54.67,48.46,42.36,36.46,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,,,,,,,,,,,...,,,,,,172.53,173.09,173.64,174.19,174.74
57,,,,,,,,,,,...,,,,,,,173.13,173.69,174.24,174.8
58,,,,,,,,,,,...,,,,,,,,173.72,174.28,174.85
59,,,,,,,,,,,...,,,,,,,,,174.31,174.88


In [ ]:
am_put = generate_american_put(spot_price, strike_price ,d, u, q, r ,num_steps)
am_put_df = pd.DataFrame(am_put)
am_put_df = am_put_df.round(2)
am_put_df.replace(0, '', inplace=True)
am_put_df.head(num_steps+1)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,47.83,41.56,35.64,30.12,25.06,20.48,16.43,12.92,9.93,7.45,...,,,,,,,,,,
1,,53.71,47.1,40.79,34.83,29.29,24.23,19.67,15.66,12.2,...,,,,,,,,,,
2,,,59.94,53.03,46.36,39.99,33.99,28.44,23.37,18.85,...,,,,,,,,,,
3,,,,66.48,59.32,52.33,45.58,39.16,33.13,27.56,...,,,,,,,,,,
4,,,,,73.29,65.92,58.67,51.6,44.78,38.31,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,,,,,,,,,,,...,,,,,,174.85,174.83,174.8,174.78,174.74
57,,,,,,,,,,,...,,,,,,,174.87,174.85,174.83,174.8
58,,,,,,,,,,,...,,,,,,,,174.88,174.87,174.85
59,,,,,,,,,,,...,,,,,,,,,174.9,174.88


In [ ]:
euro_call_binomial = euro_call_df[0][0]
am_call_binomial = am_call_df[0][0]
euro_put_binomial = euro_put_df[0][0]
am_put_binomial = am_put_df[0][0]
print(f"Binomial European Call Value: {euro_call_binomial: .3f}\nBinomial American Call Value: {am_call_binomial: .3f}")
print(f"Binomial European Put Value: {euro_put_binomial: .3f}\nBinomial American Put Value: {am_put_binomial: .3f}")

Binomial European Call Value:  83.130
Binomial American Call Value:  86.590
Binomial European Put Value:  44.750
Binomial American Put Value:  47.830


##Monte Carlo

In [ ]:
"""
spot_price = 274      # Current stock price
strike_price = 225   # Option strike price
time_to_maturity = 32/252  # Time to option expiration in years
volatility = 0.52    # Volatility of the underlying stock
risk_free_rate = 0.01 # Risk-free interest rate
dividend_yield = 0.00
num_steps = 10
"""

'\nspot_price = 274      # Current stock price\nstrike_price = 225   # Option strike price\ntime_to_maturity = 32/252  # Time to option expiration in years\nvolatility = 0.52    # Volatility of the underlying stock\nrisk_free_rate = 0.01 # Risk-free interest rate\ndividend_yield = 0.00\nnum_steps = 10\n'

In [ ]:
num_paths = 10000
simulation_price = monte_carlo_sim_option(spot_price, risk_free_rate, dividend_yield, volatility, time_to_maturity, num_paths)
Call_Expired_Value = np.maximum(0, simulation_price - strike_price)
Call_Option =  np.exp((dividend_yield -risk_free_rate)*time_to_maturity) * Call_Expired_Value.mean()
Put_Expired_Value = np.maximum(0, strike_price - simulation_price)
Put_Option =  np.exp((dividend_yield -risk_free_rate)*time_to_maturity) * Put_Expired_Value.mean()
print(f"Monte Carlo call value is {Call_Option:.2f}\nMonte Carlo put value is {Put_Option:.2f}")

Monte Carlo call value is 100.86
Monte Carlo put value is 42.75
